In [13]:
## correct net profit breakdown for NBA win, loss, cashout

import mysql.connector
from collections import defaultdict
import pandas as pd
import re

# Database connections
betting_conn = mysql.connector.connect(
    host="betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com",
    user="admin",
    password="7nRB1i2&A-K>",
    database="betting_db"
)

futures_conn = mysql.connector.connect(
    host="greenalephfutures.cnwukek8ge3b.us-east-2.rds.amazonaws.com",
    user="admin",
    password="greenalephadmin",
    database="futuresdata"
)

# Odds conversion
def american_odds_to_decimal(odds: int) -> float:
    if odds == 0:
        return 1.0
    return 1 + (odds / 100) if odds > 0 else 1 + (100 / abs(odds))

def safe_cast_odds(val):
    try:
        if isinstance(val, int):
            return val
        elif isinstance(val, float):
            return int(val)
        elif isinstance(val, str):
            val = val.strip()
            if val.lower() in ("", "na", "null", "none"):
                return 0
            match = re.search(r"[-+]?\d+", val)
            if match:
                return int(match.group())
        return 0
    except Exception:
        return 0

# Table mapping
futures_table_map = {
    ("Championship", "NBA Championship"): "NBAChampionship",
    ("Conference Winner", "Eastern Conference"): "NBAEasternConference",
    ("Conference Winner", "Western Conference"): "NBAWesternConference",
    ("Defensive Player of Year Award", "Award"): "NBADefensivePotY",
    ("Division Winner", "Atlantic Division"): "NBAAtlantic",
    ("Division Winner", "Central Division"): "NBACentral",
    ("Division Winner", "Northwest Division"): "NBANorthwest",
    ("Division Winner", "Pacific Division"): "NBAPacific",
    ("Division Winner", "Southeast Division"): "NBASoutheast",
    ("Division Winner", "Southwest Division"): "NBASouthwest",
    ("Most Improved Player Award", "Award"): "NBAMIP",
    ("Most Valuable Player Award", "Award"): "NBAMVP",
    ("Rookie of Year Award", "Award"): "NBARotY",
    ("Sixth Man of Year Award", "Award"): "NBASixthMotY",
}

team_alias_map = {
    "Philadelphia 76ers": "76ers", "Milwaukee Bucks": "Bucks", "Chicago Bulls": "Bulls",
    "Cleveland Cavaliers": "Cavaliers", "Boston Celtics": "Celtics", "Los Angeles Clippers": "Clippers",
    "Memphis Grizzlies": "Grizzlies", "Atlanta Hawks": "Hawks", "Miami Heat": "Heat",
    "Charlotte Hornets": "Hornets", "Utah Jazz": "Jazz", "Sacramento Kings": "Kings",
    "New York Knicks": "Knicks", "Los Angeles Lakers": "Lakers", "Orlando Magic": "Magic",
    "Dallas Mavericks": "Mavericks", "Brooklyn Nets": "Nets", "Denver Nuggets": "Nuggets",
    "Indiana Pacers": "Pacers", "New Orleans Pelicans": "Pelicans", "Detroit Pistons": "Pistons",
    "Toronto Raptors": "Raptors", "Houston Rockets": "Rockets", "San Antonio Spurs": "Spurs",
    "Phoenix Suns": "Suns", "Oklahoma City Thunder": "Thunder", "Minnesota Timberwolves": "Timberwolves",
    "Portland Trail Blazers": "Trail Blazers", "Golden State Warriors": "Warriors", "Washington Wizards": "Wizards",
}

# Fetch bets
cur_bets = betting_conn.cursor(dictionary=True)
cur_bets.execute("""
SELECT 
    b.WagerID, b.PotentialPayout, b.DollarsAtStake, b.NetProfit, b.WLCA, b.LegCount,
    l.LegID, l.ParticipantName, l.EventType, l.EventLabel
FROM bets b
JOIN legs l ON b.WagerID = l.WagerID
WHERE b.WhichBankroll = 'GreenAleph' AND l.LeagueName = 'NBA' AND b.WLCA IN ('Win','Loss','Cashout')
""")
rows = cur_bets.fetchall()
cur_bets.close()
betting_conn.close()

bets_dict = defaultdict(lambda: {"WLCA": None, "NetProfit": 0.0, "legs": []})
for row in rows:
    w_id = row["WagerID"]
    bets_dict[w_id]["WLCA"] = bets_dict[w_id]["WLCA"] or row["WLCA"]
    bets_dict[w_id]["NetProfit"] = bets_dict[w_id]["NetProfit"] or float(row["NetProfit"] or 0.0)
    bets_dict[w_id]["legs"].append({
        "LegID": row["LegID"],
        "EventType": row["EventType"],
        "EventLabel": row["EventLabel"],
        "ParticipantName": row["ParticipantName"]
    })

# Enhanced odds fetching function
def get_best_available_decimal_odds(event_type, event_label, participant_name):
    table = futures_table_map.get((event_type, event_label))
    if not table:
        print(f"[DEBUG] Missing table mapping for ({event_type}, {event_label})")
        return 1.0

    alias = team_alias_map.get(participant_name, participant_name)
    try:
        cur = futures_conn.cursor(dictionary=True)
        cur.execute(f"""
            SELECT date_created, BetMGM, DraftKings, Caesars, ESPNBet, FanDuel,
                   BallyBet, RiversCasino, Bet365
            FROM {table}
            WHERE team_name = %s
            ORDER BY date_created DESC
        """, (alias,))
        rows = cur.fetchall()
        cur.close()
        for row in rows:
            odds = [
                safe_cast_odds(row[col]) for col in 
                ['BetMGM', 'DraftKings', 'Caesars', 'ESPNBet', 'FanDuel', 'BallyBet', 'RiversCasino', 'Bet365']
            ]
            odds = [o for o in odds if o != 0]
            if odds:
                return american_odds_to_decimal(max(odds))
        print(f"[DEBUG] No non-zero odds found for {alias} in table {table}")
        return 1.0
    except Exception as e:
        print(f"[ERROR] Odds fetch for {alias} in {table}: {e}")
        return 1.0

# Compute proportional net profits
market_net = defaultdict(float)

for w_id, data in bets_dict.items():
    netprofit = data["NetProfit"]
    wlca = data["WLCA"]
    legs = data["legs"]

    dec_list = []
    for leg in legs:
        dec = get_best_available_decimal_odds(leg["EventType"], leg["EventLabel"], leg["ParticipantName"])
        dec_list.append((dec, leg["EventType"], leg["EventLabel"]))

    sum_excess = sum((dec - 1.0) for dec, _, _ in dec_list)
    if sum_excess <= 0:
        continue

    for dec, etype, elabel in dec_list:
        frac = (dec - 1.0) / sum_excess
        market_net[(wlca, etype, elabel)] += frac * netprofit

futures_conn.close()

# Output
df = pd.DataFrame([
    {"WLCA": k[0], "EventType": k[1], "EventLabel": k[2], "NetProfitAllocated": round(v, 2)}
    for k, v in sorted(market_net.items())
])
print(df)


[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missing table mapping for (Spread, NBA)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missing table mapping for (IST Winner, In-Season Tournament)
[DEBUG] Missing table mapping for (Group Winner, West Group Winner C)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missing table mapping for (Group Winner, East Group Winner B)
[DEBUG] Missing table mapping for (IST Winner, In-Season Tournament)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Mis

In [16]:
import mysql.connector
from collections import defaultdict
import pandas as pd
import re

# Database connections
betting_conn = mysql.connector.connect(
    host="betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com",
    user="admin",
    password="7nRB1i2&A-K>",
    database="betting_db"
)

futures_conn = mysql.connector.connect(
    host="greenalephfutures.cnwukek8ge3b.us-east-2.rds.amazonaws.com",
    user="admin",
    password="greenalephadmin",
    database="futuresdata"
)

# Odds conversion
def american_odds_to_decimal(odds: int) -> float:
    if odds == 0:
        return 1.0
    return 1 + (odds / 100) if odds > 0 else 1 + (100 / abs(odds))

def safe_cast_odds(val):
    try:
        if isinstance(val, int):
            return val
        elif isinstance(val, float):
            return int(val)
        elif isinstance(val, str):
            val = val.strip()
            if val.lower() in ("", "na", "null", "none"):
                return 0
            match = re.search(r"[-+]?\d+", val)
            if match:
                return int(match.group())
        return 0
    except Exception:
        return 0

# Table mapping
futures_table_map = {
    ("Championship", "NBA Championship"): "NBAChampionship",
    ("Conference Winner", "Eastern Conference"): "NBAEasternConference",
    ("Conference Winner", "Western Conference"): "NBAWesternConference",
    ("Defensive Player of Year Award", "Award"): "NBADefensivePotY",
    ("Division Winner", "Atlantic Division"): "NBAAtlantic",
    ("Division Winner", "Central Division"): "NBACentral",
    ("Division Winner", "Northwest Division"): "NBANorthwest",
    ("Division Winner", "Pacific Division"): "NBAPacific",
    ("Division Winner", "Southeast Division"): "NBASoutheast",
    ("Division Winner", "Southwest Division"): "NBASouthwest",
    ("Most Improved Player Award", "Award"): "NBAMIP",
    ("Most Valuable Player Award", "Award"): "NBAMVP",
    ("Rookie of Year Award", "Award"): "NBARotY",
    ("Sixth Man of Year Award", "Award"): "NBASixthMotY",
}

team_alias_map = {
    "Philadelphia 76ers": "76ers", "Milwaukee Bucks": "Bucks", "Chicago Bulls": "Bulls",
    "Cleveland Cavaliers": "Cavaliers", "Boston Celtics": "Celtics", "Los Angeles Clippers": "Clippers",
    "Memphis Grizzlies": "Grizzlies", "Atlanta Hawks": "Hawks", "Miami Heat": "Heat",
    "Charlotte Hornets": "Hornets", "Utah Jazz": "Jazz", "Sacramento Kings": "Kings",
    "New York Knicks": "Knicks", "Los Angeles Lakers": "Lakers", "Orlando Magic": "Magic",
    "Dallas Mavericks": "Mavericks", "Brooklyn Nets": "Nets", "Denver Nuggets": "Nuggets",
    "Indiana Pacers": "Pacers", "New Orleans Pelicans": "Pelicans", "Detroit Pistons": "Pistons",
    "Toronto Raptors": "Raptors", "Houston Rockets": "Rockets", "San Antonio Spurs": "Spurs",
    "Phoenix Suns": "Suns", "Oklahoma City Thunder": "Thunder", "Minnesota Timberwolves": "Timberwolves",
    "Portland Trail Blazers": "Trail Blazers", "Golden State Warriors": "Warriors", "Washington Wizards": "Wizards",
}

# Fetch bets with WLCA = Active
cur_bets = betting_conn.cursor(dictionary=True)
cur_bets.execute("""
SELECT 
    b.WagerID, b.PotentialPayout, b.DollarsAtStake, b.WLCA, b.LegCount,
    l.LegID, l.ParticipantName, l.EventType, l.EventLabel
FROM bets b
JOIN legs l ON b.WagerID = l.WagerID
WHERE b.WhichBankroll = 'GreenAleph' AND l.LeagueName = 'NBA' AND b.WLCA = 'Active'
""")
rows = cur_bets.fetchall()
cur_bets.close()
betting_conn.close()

bets_dict = defaultdict(lambda: {"DollarsAtStake": 0.0, "legs": []})
for row in rows:
    w_id = row["WagerID"]
    bets_dict[w_id]["DollarsAtStake"] = bets_dict[w_id]["DollarsAtStake"] or float(row["DollarsAtStake"] or 0.0)
    bets_dict[w_id]["legs"].append({
        "LegID": row["LegID"],
        "EventType": row["EventType"],
        "EventLabel": row["EventLabel"],
        "ParticipantName": row["ParticipantName"]
    })

# Odds fetcher: best non-zero American odds converted to decimal
def get_best_available_decimal_odds(event_type, event_label, participant_name):
    table = futures_table_map.get((event_type, event_label))
    if not table:
        print(f"[DEBUG] Missing table mapping for ({event_type}, {event_label})")
        return 1.0

    alias = team_alias_map.get(participant_name, participant_name)
    try:
        cur = futures_conn.cursor(dictionary=True)
        cur.execute(f"""
            SELECT date_created, BetMGM, DraftKings, Caesars, ESPNBet, FanDuel,
                   BallyBet, RiversCasino, Bet365
            FROM {table}
            WHERE team_name = %s
            ORDER BY date_created DESC
        """, (alias,))
        rows = cur.fetchall()
        cur.close()
        for row in rows:
            odds = [
                safe_cast_odds(row[col]) for col in 
                ['BetMGM', 'DraftKings', 'Caesars', 'ESPNBet', 'FanDuel', 'BallyBet', 'RiversCasino', 'Bet365']
            ]
            odds = [o for o in odds if o != 0]
            if odds:
                return american_odds_to_decimal(max(odds))
        print(f"[DEBUG] No non-zero odds found for {alias} in table {table}")
        return 1.0
    except Exception as e:
        print(f"[ERROR] Odds fetch for {alias} in {table}: {e}")
        return 1.0

# Allocate stake proportionally
stake_alloc = defaultdict(float)

for w_id, data in bets_dict.items():
    stake = data["DollarsAtStake"]
    legs = data["legs"]

    dec_list = []
    for leg in legs:
        dec = get_best_available_decimal_odds(leg["EventType"], leg["EventLabel"], leg["ParticipantName"])
        dec_list.append((dec, leg["EventType"], leg["EventLabel"]))

    sum_excess = sum((dec - 1.0) for dec, _, _ in dec_list)
    if sum_excess <= 0:
        continue

    for dec, etype, elabel in dec_list:
        frac = (dec - 1.0) / sum_excess
        stake_alloc[(etype, elabel)] += frac * stake

futures_conn.close()

# Output
df = pd.DataFrame([
    {"EventType": k[0], "EventLabel": k[1], "ActiveStakeAllocated": round(v, 2)}
    for k, v in sorted(stake_alloc.items())
])
print(df)


[DEBUG] Missing table mapping for (Coach of Year Award, Award)
[DEBUG] Missing table mapping for (To make Playoffs, Yes)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (To make Playoffs, Yes)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (To make Playoffs, Yes)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (To make Playoffs, Yes)
[DEBUG] Missing table mapping for (To make Playoffs, Yes)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] No non-zero odds found for GS Warriors in table NBAWesternConference
[DEBUG] No non-zero odds found for Jordan Hawkins in table NBASixthMotY
[DEBUG] Missing table mapping for (Coach of Year Award, Award)
[DEBUG] Missing table mapping for (C

[DEBUG] No non-zero odds found for Tidjane Salaun in table NBARotY
[DEBUG] No non-zero odds found for Tidjane Salaun in table NBARotY
[DEBUG] No non-zero odds found for Josh Giddey in table NBAMIP
[DEBUG] No non-zero odds found for DeAndre Ayton in table NBAMIP
[DEBUG] No non-zero odds found for Josh Giddey in table NBAMIP
[DEBUG] No non-zero odds found for DeAndre Ayton in table NBAMIP
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (To make Playoffs, No)
                         EventType              EventLabel  \
0                     Championship        NBA Championship   
1      Clutch Player of Year Award                   Award   
2              Coach of Year Award                   Award   
3                Conference Winner      Eastern Conference   
4                Conference Winner      Western Conference   
5   Defensive Player of Year Award                   Award   
6                  Division Winner       Atlantic Division   


In [17]:
import mysql.connector
from collections import defaultdict
import pandas as pd
import re

# Database connections
betting_conn = mysql.connector.connect(
    host="betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com",
    user="admin",
    password="7nRB1i2&A-K>",
    database="betting_db"
)

futures_conn = mysql.connector.connect(
    host="greenalephfutures.cnwukek8ge3b.us-east-2.rds.amazonaws.com",
    user="admin",
    password="greenalephadmin",
    database="futuresdata"
)

# Odds conversion
def american_odds_to_decimal(odds: int) -> float:
    if odds == 0:
        return 1.0
    return 1 + (odds / 100) if odds > 0 else 1 + (100 / abs(odds))

def american_odds_to_probability(odds: int) -> float:
    if odds == 0:
        return 0.0
    return 100 / (odds + 100) if odds > 0 else abs(odds) / (abs(odds) + 100)

def safe_cast_odds(val):
    try:
        if isinstance(val, int):
            return val
        elif isinstance(val, float):
            return int(val)
        elif isinstance(val, str):
            val = val.strip()
            if val.lower() in ("", "na", "null", "none"):
                return 0
            match = re.search(r"[-+]?\d+", val)
            if match:
                return int(match.group())
        return 0
    except Exception:
        return 0

# Table mapping
futures_table_map = {
    ("Championship", "NBA Championship"): "NBAChampionship",
    ("Conference Winner", "Eastern Conference"): "NBAEasternConference",
    ("Conference Winner", "Western Conference"): "NBAWesternConference",
    ("Defensive Player of Year Award", "Award"): "NBADefensivePotY",
    ("Division Winner", "Atlantic Division"): "NBAAtlantic",
    ("Division Winner", "Central Division"): "NBACentral",
    ("Division Winner", "Northwest Division"): "NBANorthwest",
    ("Division Winner", "Pacific Division"): "NBAPacific",
    ("Division Winner", "Southeast Division"): "NBASoutheast",
    ("Division Winner", "Southwest Division"): "NBASouthwest",
    ("Most Improved Player Award", "Award"): "NBAMIP",
    ("Most Valuable Player Award", "Award"): "NBAMVP",
    ("Rookie of Year Award", "Award"): "NBARotY",
    ("Sixth Man of Year Award", "Award"): "NBASixthMotY",
}

team_alias_map = {
    "Philadelphia 76ers": "76ers", "Milwaukee Bucks": "Bucks", "Chicago Bulls": "Bulls",
    "Cleveland Cavaliers": "Cavaliers", "Boston Celtics": "Celtics", "Los Angeles Clippers": "Clippers",
    "Memphis Grizzlies": "Grizzlies", "Atlanta Hawks": "Hawks", "Miami Heat": "Heat",
    "Charlotte Hornets": "Hornets", "Utah Jazz": "Jazz", "Sacramento Kings": "Kings",
    "New York Knicks": "Knicks", "Los Angeles Lakers": "Lakers", "Orlando Magic": "Magic",
    "Dallas Mavericks": "Mavericks", "Brooklyn Nets": "Nets", "Denver Nuggets": "Nuggets",
    "Indiana Pacers": "Pacers", "New Orleans Pelicans": "Pelicans", "Detroit Pistons": "Pistons",
    "Toronto Raptors": "Raptors", "Houston Rockets": "Rockets", "San Antonio Spurs": "Spurs",
    "Phoenix Suns": "Suns", "Oklahoma City Thunder": "Thunder", "Minnesota Timberwolves": "Timberwolves",
    "Portland Trail Blazers": "Trail Blazers", "Golden State Warriors": "Warriors", "Washington Wizards": "Wizards",
}

# Fetch active bets
cur_bets = betting_conn.cursor(dictionary=True)
cur_bets.execute("""
SELECT 
    b.WagerID, b.PotentialPayout, b.DollarsAtStake, b.WLCA, b.LegCount,
    l.LegID, l.ParticipantName, l.EventType, l.EventLabel
FROM bets b
JOIN legs l ON b.WagerID = l.WagerID
WHERE b.WhichBankroll = 'GreenAleph' AND l.LeagueName = 'NBA' AND b.WLCA = 'Active'
""")
rows = cur_bets.fetchall()
cur_bets.close()
betting_conn.close()

bets_dict = defaultdict(lambda: {"PotentialPayout": 0.0, "legs": []})
for row in rows:
    w_id = row["WagerID"]
    bets_dict[w_id]["PotentialPayout"] = bets_dict[w_id]["PotentialPayout"] or float(row["PotentialPayout"] or 0.0)
    bets_dict[w_id]["legs"].append({
        "LegID": row["LegID"],
        "EventType": row["EventType"],
        "EventLabel": row["EventLabel"],
        "ParticipantName": row["ParticipantName"]
    })

# Fetch best available decimal odds and probability
def get_best_decimal_and_probability(event_type, event_label, participant_name):
    table = futures_table_map.get((event_type, event_label))
    if not table:
        print(f"[DEBUG] Missing table mapping for ({event_type}, {event_label})")
        return 1.0, 0.0

    alias = team_alias_map.get(participant_name, participant_name)
    try:
        cur = futures_conn.cursor(dictionary=True)
        cur.execute(f"""
            SELECT date_created, BetMGM, DraftKings, Caesars, ESPNBet, FanDuel,
                   BallyBet, RiversCasino, Bet365
            FROM {table}
            WHERE team_name = %s
            ORDER BY date_created DESC
        """, (alias,))
        rows = cur.fetchall()
        cur.close()
        for row in rows:
            all_odds = [
                safe_cast_odds(row[col]) for col in 
                ['BetMGM', 'DraftKings', 'Caesars', 'ESPNBet', 'FanDuel', 'BallyBet', 'RiversCasino', 'Bet365']
            ]
            odds = [o for o in all_odds if o != 0]
            if odds:
                best = max(odds)
                return american_odds_to_decimal(best), american_odds_to_probability(best)
        print(f"[DEBUG] No non-zero odds found for {alias} in table {table}")
        return 1.0, 0.0
    except Exception as e:
        print(f"[ERROR] Odds fetch for {alias} in {table}: {e}")
        return 1.0, 0.0

# Proportional allocation of expected payout
market_expected = defaultdict(float)

for w_id, data in bets_dict.items():
    pot = data["PotentialPayout"]
    legs = data["legs"]

    dec_probs = []
    parlay_prob = 1.0
    for leg in legs:
        dec, prob = get_best_decimal_and_probability(leg["EventType"], leg["EventLabel"], leg["ParticipantName"])
        dec_probs.append((dec, leg["EventType"], leg["EventLabel"]))
        parlay_prob *= prob

    if parlay_prob == 0:
        continue

    expected_payout = pot * parlay_prob
    sum_excess = sum((dec - 1.0) for dec, _, _ in dec_probs)
    if sum_excess <= 0:
        continue

    for dec, etype, elabel in dec_probs:
        frac = (dec - 1.0) / sum_excess
        market_expected[(etype, elabel)] += frac * expected_payout

futures_conn.close()

# Output results
df = pd.DataFrame([
    {"EventType": k[0], "EventLabel": k[1], "ActiveExpectedPayout": round(v, 2)}
    for k, v in sorted(market_expected.items())
])
print(df)


[DEBUG] Missing table mapping for (Coach of Year Award, Award)
[DEBUG] Missing table mapping for (To make Playoffs, Yes)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (To make Playoffs, Yes)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (To make Playoffs, Yes)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (To make Playoffs, Yes)
[DEBUG] Missing table mapping for (To make Playoffs, Yes)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] No non-zero odds found for GS Warriors in table NBAWesternConference
[DEBUG] No non-zero odds found for Jordan Hawkins in table NBASixthMotY
[DEBUG] Missing table mapping for (Coach of Year Award, Award)
[DEBUG] Missing table mapping for (C

[DEBUG] No non-zero odds found for Tidjane Salaun in table NBARotY
[DEBUG] No non-zero odds found for Tidjane Salaun in table NBARotY
[DEBUG] No non-zero odds found for Josh Giddey in table NBAMIP
[DEBUG] No non-zero odds found for DeAndre Ayton in table NBAMIP
[DEBUG] No non-zero odds found for Josh Giddey in table NBAMIP
[DEBUG] No non-zero odds found for DeAndre Ayton in table NBAMIP
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (To make Playoffs, No)
                         EventType          EventLabel  ActiveExpectedPayout
0                     Championship    NBA Championship              42344.94
1                Conference Winner  Eastern Conference               1712.38
2                Conference Winner  Western Conference              29268.87
3   Defensive Player of Year Award               Award              20865.52
4                  Division Winner   Atlantic Division               9167.48
5                  Division Winner

In [18]:
import mysql.connector
from collections import defaultdict
import pandas as pd
import re

###############################################################################
# 1) Connect to Databases
###############################################################################
betting_conn = mysql.connector.connect(
    host="betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com",
    user="admin",
    password="7nRB1i2&A-K>",
    database="betting_db"
)

futures_conn = mysql.connector.connect(
    host="greenalephfutures.cnwukek8ge3b.us-east-2.rds.amazonaws.com",
    user="admin",
    password="greenalephadmin",
    database="futuresdata"
)

###############################################################################
# 2) Odds Conversion Helpers
###############################################################################
def american_odds_to_decimal(odds: int) -> float:
    """Convert American odds to decimal odds."""
    if odds == 0:
        return 1.0
    return 1.0 + (odds / 100.0) if odds > 0 else 1.0 + (100.0 / abs(odds))

def american_odds_to_probability(odds: int) -> float:
    """Convert American odds to implied probability (0..1)."""
    if odds == 0:
        return 0.0
    return 100.0 / (odds + 100.0) if odds > 0 else float(abs(odds)) / (abs(odds) + 100.0)

def safe_cast_odds(val):
    """Safely convert the DB odds field to int, handling strings, floats, etc."""
    try:
        if isinstance(val, int):
            return val
        elif isinstance(val, float):
            return int(val)
        elif isinstance(val, str):
            val = val.strip()
            if val.lower() in ("", "na", "null", "none"):
                return 0
            match = re.search(r"[-+]?\d+", val)
            if match:
                return int(match.group())
        return 0
    except Exception:
        return 0

###############################################################################
# 3) Table & Team Mappings
###############################################################################
futures_table_map = {
    ("Championship", "NBA Championship"): "NBAChampionship",
    ("Conference Winner", "Eastern Conference"): "NBAEasternConference",
    ("Conference Winner", "Western Conference"): "NBAWesternConference",
    ("Defensive Player of Year Award", "Award"): "NBADefensivePotY",
    ("Division Winner", "Atlantic Division"): "NBAAtlantic",
    ("Division Winner", "Central Division"): "NBACentral",
    ("Division Winner", "Northwest Division"): "NBANorthwest",
    ("Division Winner", "Pacific Division"): "NBAPacific",
    ("Division Winner", "Southeast Division"): "NBASoutheast",
    ("Division Winner", "Southwest Division"): "NBASouthwest",
    ("Most Improved Player Award", "Award"): "NBAMIP",
    ("Most Valuable Player Award", "Award"): "NBAMVP",
    ("Rookie of Year Award", "Award"): "NBARotY",
    ("Sixth Man of Year Award", "Award"): "NBASixthMotY",
}

team_alias_map = {
    "Philadelphia 76ers": "76ers", "Milwaukee Bucks": "Bucks", "Chicago Bulls": "Bulls",
    "Cleveland Cavaliers": "Cavaliers", "Boston Celtics": "Celtics", "Los Angeles Clippers": "Clippers",
    "Memphis Grizzlies": "Grizzlies", "Atlanta Hawks": "Hawks", "Miami Heat": "Heat",
    "Charlotte Hornets": "Hornets", "Utah Jazz": "Jazz", "Sacramento Kings": "Kings",
    "New York Knicks": "Knicks", "Los Angeles Lakers": "Lakers", "Orlando Magic": "Magic",
    "Dallas Mavericks": "Mavericks", "Brooklyn Nets": "Nets", "Denver Nuggets": "Nuggets",
    "Indiana Pacers": "Pacers", "New Orleans Pelicans": "Pelicans", "Detroit Pistons": "Pistons",
    "Toronto Raptors": "Raptors", "Houston Rockets": "Rockets", "San Antonio Spurs": "Spurs",
    "Phoenix Suns": "Suns", "Oklahoma City Thunder": "Thunder", "Minnesota Timberwolves": "Timberwolves",
    "Portland Trail Blazers": "Trail Blazers", "Golden State Warriors": "Warriors", "Washington Wizards": "Wizards",
}

sportsbook_cols = ["BetMGM","DraftKings","Caesars","ESPNBet","FanDuel","BallyBet","RiversCasino","Bet365"]

###############################################################################
# 4) Best Available Odds Function
###############################################################################
def get_best_decimal_and_probability(event_type, event_label, participant_name):
    """
    Return (decimal_odds, probability) from the maximum american odds
    among all non-zero fields on the most recent date_created.
    """
    table = futures_table_map.get((event_type, event_label))
    if not table:
        # No recognized futures table => default
        print(f"[DEBUG] Missing table mapping for ({event_type}, {event_label})")
        return 1.0, 0.0

    alias = team_alias_map.get(participant_name, participant_name)
    try:
        c = futures_conn.cursor(dictionary=True)
        # Grab all rows for this participant, newest first
        c.execute(f"""
            SELECT date_created, {','.join(sportsbook_cols)}
            FROM {table}
            WHERE team_name=%s
            ORDER BY date_created DESC
        """, (alias,))
        all_rows = c.fetchall()
        c.close()

        # For each row in descending date order:
        #  find the max non-zero american odds. If any exist, use that
        for row in all_rows:
            # cast each field safely
            numeric_odds = [safe_cast_odds(row[col]) for col in sportsbook_cols]
            # filter out zero
            numeric_odds = [o for o in numeric_odds if o != 0]
            if numeric_odds:
                best = max(numeric_odds)
                return (american_odds_to_decimal(best),
                        american_odds_to_probability(best))

        print(f"[DEBUG] No non-zero odds found for {alias} in {table}")
        return 1.0, 0.0

    except Exception as e:
        print(f"[ERROR] Odds fetch for {alias} in {table}: {e}")
        return 1.0, 0.0

###############################################################################
# 5) Query #1: Active => DollarsAtStake, ExpectedPayout
###############################################################################
# We'll store results in two dicts: active_stake_dict, active_exp_dict
active_stake_dict = defaultdict(float)
active_exp_dict   = defaultdict(float)

c_active = betting_conn.cursor(dictionary=True)
c_active.execute("""
SELECT 
    b.WagerID, b.PotentialPayout, b.DollarsAtStake, b.WLCA, b.LegCount,
    l.LegID, l.ParticipantName, l.EventType, l.EventLabel
FROM bets b
JOIN legs l ON b.WagerID = l.WagerID
WHERE b.WhichBankroll='GreenAleph'
  AND l.LeagueName='NBA'
  AND b.WLCA='Active'
""")
rows_active = c_active.fetchall()
c_active.close()

# We'll group the active bets by WagerID
active_bets = defaultdict(lambda: {
    "PotentialPayout": 0.0,
    "DollarsAtStake": 0.0,
    "legs": []
})
for row in rows_active:
    wid = row["WagerID"]
    ab = active_bets[wid]
    if ab["PotentialPayout"] == 0.0:
        ab["PotentialPayout"] = float(row["PotentialPayout"] or 0.0)
    if ab["DollarsAtStake"] == 0.0:
        ab["DollarsAtStake"] = float(row["DollarsAtStake"] or 0.0)
    ab["legs"].append({
        "EventType": row["EventType"],
        "EventLabel": row["EventLabel"],
        "ParticipantName": row["ParticipantName"]
    })

# For each active bet, we compute:
#  - parlay probability => expected payout = pot * parlay_prob
#  - sum_excess => fraction_i => allocate stake & exp to each (etype,elabel)
for wid, data in active_bets.items():
    pot = data["PotentialPayout"]
    stake = data["DollarsAtStake"]
    legs = data["legs"]

    dec_probs = []
    parlay_prob = 1.0
    for leg in legs:
        dec, prob = get_best_decimal_and_probability(
            leg["EventType"], leg["EventLabel"], leg["ParticipantName"]
        )
        parlay_prob *= prob
        dec_probs.append((dec, leg["EventType"], leg["EventLabel"]))

    if parlay_prob == 0:
        # zero prob => skip
        continue

    expected_payout = pot * parlay_prob
    sum_excess = sum((d - 1.0) for d, _, _ in dec_probs)
    if sum_excess <= 0:
        # can't do fraction approach
        continue

    for dec, etype, elabel in dec_probs:
        frac = (dec - 1.0)/sum_excess
        # allocate stake
        active_stake_dict[(etype, elabel)] += frac * stake
        # allocate expected payout
        active_exp_dict[(etype, elabel)] += frac * expected_payout

###############################################################################
# 6) Query #2: Realized => NetProfit for WLCA in (Win,Loss,Cashout)
###############################################################################
realized_np_dict = defaultdict(float)

c_real = betting_conn.cursor(dictionary=True)
c_real.execute("""
SELECT
    b.WagerID, b.NetProfit, b.WLCA, b.LegCount,
    l.LegID, l.ParticipantName, l.EventType, l.EventLabel
FROM bets b
JOIN legs l ON b.WagerID = l.WagerID
WHERE b.WhichBankroll='GreenAleph'
  AND l.LeagueName='NBA'
  AND b.WLCA IN ('Win','Loss','Cashout')
""")
rows_real = c_real.fetchall()
c_real.close()
betting_conn.close()

real_bets = defaultdict(lambda: {"WLCA": None, "NetProfit":0.0, "legs":[]})
for row in rows_real:
    wid = row["WagerID"]
    rb = real_bets[wid]
    if rb["WLCA"] is None:
        rb["WLCA"] = row["WLCA"]
    if rb["NetProfit"] == 0.0 and row["NetProfit"] is not None:
        rb["NetProfit"] = float(row["NetProfit"])
    rb["legs"].append({
        "EventType": row["EventType"],
        "EventLabel": row["EventLabel"],
        "ParticipantName": row["ParticipantName"]
    })

# For each bet => net profit => allocate proportionally
for wid, info in real_bets.items():
    netp = info["NetProfit"]
    legs = info["legs"]
    wlca = info["WLCA"]

    dec_list = []
    for leg in legs:
        dec, prob = get_best_decimal_and_probability(
            leg["EventType"], leg["EventLabel"], leg["ParticipantName"]
        )
        dec_list.append((dec, leg["EventType"], leg["EventLabel"]))

    sum_excess = sum((d - 1.0) for d,_,_ in dec_list)
    if sum_excess<=0:
        continue

    for dec, etype, elabel in dec_list:
        frac = (dec - 1.0)/sum_excess
        realized_np_dict[(etype, elabel)] += frac*netp

futures_conn.close()

###############################################################################
# 7) Combine Results
###############################################################################
results_map = {}
all_keys = set(list(active_stake_dict.keys()) +
               list(active_exp_dict.keys()) +
               list(realized_np_dict.keys()))

for key in all_keys:
    # key is (EventType, EventLabel)
    results_map[key] = {
        "EventType": key[0],
        "EventLabel": key[1],
        "ActiveDollarsAtStake": round(active_stake_dict.get(key, 0.0),2),
        "ActiveExpectedPayout": round(active_exp_dict.get(key, 0.0),2),
        "RealizedNetProfit": round(realized_np_dict.get(key,0.0),2)
    }

df = pd.DataFrame(results_map.values()).sort_values(["EventType","EventLabel"])
print(df)

# Optional: df.to_csv("nba_allocation_results.csv", index=False)


[DEBUG] Missing table mapping for (Coach of Year Award, Award)
[DEBUG] Missing table mapping for (To make Playoffs, Yes)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (To make Playoffs, Yes)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (To make Playoffs, Yes)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (To make Playoffs, Yes)
[DEBUG] Missing table mapping for (To make Playoffs, Yes)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] No non-zero odds found for GS Warriors in NBAWesternConference
[DEBUG] No non-zero odds found for Jordan Hawkins in NBASixthMotY
[DEBUG] Missing table mapping for (Coach of Year Award, Award)
[DEBUG] Missing table mapping for (Coach of Year

[DEBUG] No non-zero odds found for DeAndre Ayton in NBAMIP
[DEBUG] No non-zero odds found for Josh Giddey in NBAMIP
[DEBUG] No non-zero odds found for DeAndre Ayton in NBAMIP
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missing table mapping for (Spread, NBA)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missing table mapping for (IST Winner, In-Season Tournament)
[DEBUG] Missing table mapping for (Group Winner, West Group Winner C)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missin

In [19]:
# ActiveDollarsAtStake, ActiveExpectedPayout, RealizedNetProfit, and ExpectedValue

import mysql.connector
from collections import defaultdict
import pandas as pd
import re

# 1) Database Connections
betting_conn = mysql.connector.connect(
    host="betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com",
    user="admin",
    password="7nRB1i2&A-K>",
    database="betting_db"
)

futures_conn = mysql.connector.connect(
    host="greenalephfutures.cnwukek8ge3b.us-east-2.rds.amazonaws.com",
    user="admin",
    password="greenalephadmin",
    database="futuresdata"
)

# 2) Odds Conversion & Helpers
def american_odds_to_decimal(odds: int) -> float:
    if odds == 0:
        return 1.0
    return 1.0 + (odds / 100.0) if odds > 0 else 1.0 + (100.0 / abs(odds))

def american_odds_to_probability(odds: int) -> float:
    if odds == 0:
        return 0.0
    return 100.0 / (odds + 100.0) if odds > 0 else float(abs(odds)) / (abs(odds) + 100.0)

def safe_cast_odds(val):
    try:
        if isinstance(val, int):
            return val
        elif isinstance(val, float):
            return int(val)
        elif isinstance(val, str):
            val = val.strip()
            if val.lower() in ("", "na", "null", "none"):
                return 0
            match = re.search(r"[-+]?\d+", val)
            if match:
                return int(match.group())
        return 0
    except Exception:
        return 0

# 3) Table & Team Mappings
futures_table_map = {
    ("Championship", "NBA Championship"): "NBAChampionship",
    ("Conference Winner", "Eastern Conference"): "NBAEasternConference",
    ("Conference Winner", "Western Conference"): "NBAWesternConference",
    ("Defensive Player of Year Award", "Award"): "NBADefensivePotY",
    ("Division Winner", "Atlantic Division"): "NBAAtlantic",
    ("Division Winner", "Central Division"): "NBACentral",
    ("Division Winner", "Northwest Division"): "NBANorthwest",
    ("Division Winner", "Pacific Division"): "NBAPacific",
    ("Division Winner", "Southeast Division"): "NBASoutheast",
    ("Division Winner", "Southwest Division"): "NBASouthwest",
    ("Most Improved Player Award", "Award"): "NBAMIP",
    ("Most Valuable Player Award", "Award"): "NBAMVP",
    ("Rookie of Year Award", "Award"): "NBARotY",
    ("Sixth Man of Year Award", "Award"): "NBASixthMotY",
}

team_alias_map = {
    "Philadelphia 76ers": "76ers",
    "Milwaukee Bucks": "Bucks",
    "Chicago Bulls": "Bulls",
    "Cleveland Cavaliers": "Cavaliers",
    "Boston Celtics": "Celtics",
    "Los Angeles Clippers": "Clippers",
    "Memphis Grizzlies": "Grizzlies",
    "Atlanta Hawks": "Hawks",
    "Miami Heat": "Heat",
    "Charlotte Hornets": "Hornets",
    "Utah Jazz": "Jazz",
    "Sacramento Kings": "Kings",
    "New York Knicks": "Knicks",
    "Los Angeles Lakers": "Lakers",
    "Orlando Magic": "Magic",
    "Dallas Mavericks": "Mavericks",
    "Brooklyn Nets": "Nets",
    "Denver Nuggets": "Nuggets",
    "Indiana Pacers": "Pacers",
    "New Orleans Pelicans": "Pelicans",
    "Detroit Pistons": "Pistons",
    "Toronto Raptors": "Raptors",
    "Houston Rockets": "Rockets",
    "San Antonio Spurs": "Spurs",
    "Phoenix Suns": "Suns",
    "Oklahoma City Thunder": "Thunder",
    "Minnesota Timberwolves": "Timberwolves",
    "Portland Trail Blazers": "Trail Blazers",
    "Golden State Warriors": "Warriors",
    "Washington Wizards": "Wizards",
}

sportsbook_cols = ["BetMGM","DraftKings","Caesars","ESPNBet","FanDuel","BallyBet","RiversCasino","Bet365"]

# 4) Best available odds from non-zero fields
def get_best_decimal_and_probability(event_type, event_label, participant_name):
    table = futures_table_map.get((event_type, event_label))
    if not table:
        print(f"[DEBUG] Missing table mapping for ({event_type}, {event_label})")
        return 1.0, 0.0

    alias = team_alias_map.get(participant_name, participant_name)
    try:
        cur = futures_conn.cursor(dictionary=True)
        cur.execute(f"""
            SELECT date_created, {','.join(sportsbook_cols)}
            FROM {table}
            WHERE team_name = %s
            ORDER BY date_created DESC
        """, (alias,))
        all_rows = cur.fetchall()
        cur.close()

        for row in all_rows:
            numeric_odds = [safe_cast_odds(row[col]) for col in sportsbook_cols]
            numeric_odds = [o for o in numeric_odds if o != 0]
            if numeric_odds:
                best = max(numeric_odds)
                return (american_odds_to_decimal(best),
                        american_odds_to_probability(best))
        print(f"[DEBUG] No non-zero odds found for {alias} in table {table}")
        return 1.0, 0.0
    except Exception as e:
        print(f"[ERROR] Odds fetch for {alias} in {table}: {e}")
        return 1.0, 0.0

###############################################################################
# 5)  Query #1: Active => DollarsAtStake, ExpectedPayout
###############################################################################
active_stake_dict = defaultdict(float)
active_exp_dict   = defaultdict(float)

c_active = betting_conn.cursor(dictionary=True)
c_active.execute("""
SELECT
    b.WagerID, b.PotentialPayout, b.DollarsAtStake, b.WLCA, b.LegCount,
    l.LegID, l.ParticipantName, l.EventType, l.EventLabel
FROM bets b
JOIN legs l ON b.WagerID = l.WagerID
WHERE b.WhichBankroll='GreenAleph'
  AND l.LeagueName='NBA'
  AND b.WLCA='Active'
""")
rows_active = c_active.fetchall()
c_active.close()

active_bets = defaultdict(lambda: {
    "PotentialPayout": 0.0,
    "DollarsAtStake": 0.0,
    "legs": []
})
for row in rows_active:
    wid = row["WagerID"]
    ab = active_bets[wid]
    if ab["PotentialPayout"] == 0.0:
        ab["PotentialPayout"] = float(row["PotentialPayout"] or 0.0)
    if ab["DollarsAtStake"] == 0.0:
        ab["DollarsAtStake"] = float(row["DollarsAtStake"] or 0.0)
    ab["legs"].append({
        "EventType": row["EventType"],
        "EventLabel": row["EventLabel"],
        "ParticipantName": row["ParticipantName"]
    })

for wid, data in active_bets.items():
    pot   = data["PotentialPayout"]
    stake = data["DollarsAtStake"]
    legs  = data["legs"]

    dec_probs = []
    parlay_prob = 1.0
    for leg in legs:
        dec, prob = get_best_decimal_and_probability(
            leg["EventType"], leg["EventLabel"], leg["ParticipantName"]
        )
        parlay_prob *= prob
        dec_probs.append((dec, leg["EventType"], leg["EventLabel"]))

    if parlay_prob == 0:
        continue

    expected_payout = pot * parlay_prob
    sum_excess = sum((d - 1.0) for d,_,_ in dec_probs)
    if sum_excess <= 0:
        continue

    for dec, etype, elabel in dec_probs:
        frac = (dec - 1.0)/sum_excess
        active_stake_dict[(etype, elabel)] += frac * stake
        active_exp_dict[(etype, elabel)]   += frac * expected_payout

###############################################################################
# 6) Query #2: Realized => NetProfit for WLCA in (Win,Loss,Cashout)
###############################################################################
realized_np_dict = defaultdict(float)

c_real = betting_conn.cursor(dictionary=True)
c_real.execute("""
SELECT
    b.WagerID, b.NetProfit, b.WLCA, b.LegCount,
    l.LegID, l.ParticipantName, l.EventType, l.EventLabel
FROM bets b
JOIN legs l ON b.WagerID = l.WagerID
WHERE b.WhichBankroll='GreenAleph'
  AND l.LeagueName='NBA'
  AND b.WLCA IN ('Win','Loss','Cashout')
""")
rows_real = c_real.fetchall()
c_real.close()
betting_conn.close()

real_bets = defaultdict(lambda: {"WLCA": None, "NetProfit": 0.0, "legs":[]})
for row in rows_real:
    wid = row["WagerID"]
    rb = real_bets[wid]
    if rb["WLCA"] is None:
        rb["WLCA"] = row["WLCA"]
    if rb["NetProfit"] == 0.0 and row["NetProfit"] is not None:
        rb["NetProfit"] = float(row["NetProfit"] or 0.0)
    rb["legs"].append({
        "EventType": row["EventType"],
        "EventLabel": row["EventLabel"],
        "ParticipantName": row["ParticipantName"]
    })

for wid, info in real_bets.items():
    netp = info["NetProfit"]
    legs = info["legs"]

    dec_list = []
    for leg in legs:
        dec, prob = get_best_decimal_and_probability(
            leg["EventType"], leg["EventLabel"], leg["ParticipantName"]
        )
        dec_list.append((dec, leg["EventType"], leg["EventLabel"]))

    sum_excess = sum((d - 1.0) for d,_,_ in dec_list)
    if sum_excess<=0:
        continue

    for dec, etype, elabel in dec_list:
        frac = (dec - 1.0)/sum_excess
        realized_np_dict[(etype, elabel)] += frac*netp

futures_conn.close()

###############################################################################
# 7) Combine results & compute `ExpectedValue`
#    = ActiveExpectedPayout - ActiveDollarsAtStake + RealizedNetProfit
###############################################################################
results_map = {}
all_keys = set(list(active_stake_dict.keys()) +
               list(active_exp_dict.keys()) +
               list(realized_np_dict.keys()))

for key in all_keys:
    stake_val  = active_stake_dict.get(key, 0.0)
    exp_val    = active_exp_dict.get(key, 0.0)
    real_net   = realized_np_dict.get(key, 0.0)
    # compute
    exp_value  = exp_val - stake_val + real_net
    results_map[key] = {
        "EventType": key[0],
        "EventLabel": key[1],
        "ActiveDollarsAtStake": round(stake_val,2),
        "ActiveExpectedPayout": round(exp_val,2),
        "RealizedNetProfit": round(real_net,2),
        "ExpectedValue": round(exp_value,2),
    }

df = pd.DataFrame(results_map.values()).sort_values(["EventType","EventLabel"])

# Expand cell display to 'full screen' width
pd.set_option('display.width', None)        # or a large integer, e.g. 2000
pd.set_option('display.max_colwidth', None) # so columns can be fully visible

print(df)


[DEBUG] Missing table mapping for (Coach of Year Award, Award)
[DEBUG] Missing table mapping for (To make Playoffs, Yes)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (To make Playoffs, Yes)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (To make Playoffs, Yes)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (To make Playoffs, Yes)
[DEBUG] Missing table mapping for (To make Playoffs, Yes)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] No non-zero odds found for GS Warriors in table NBAWesternConference
[DEBUG] No non-zero odds found for Jordan Hawkins in table NBASixthMotY
[DEBUG] Missing table mapping for (Coach of Year Award, Award)
[DEBUG] Missing table mapping for (C

[DEBUG] No non-zero odds found for Tidjane Salaun in table NBARotY
[DEBUG] No non-zero odds found for Tidjane Salaun in table NBARotY
[DEBUG] No non-zero odds found for Josh Giddey in table NBAMIP
[DEBUG] No non-zero odds found for DeAndre Ayton in table NBAMIP
[DEBUG] No non-zero odds found for Josh Giddey in table NBAMIP
[DEBUG] No non-zero odds found for DeAndre Ayton in table NBAMIP
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (To make Playoffs, No)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missing table mapping for (Spread, NBA)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missing table mapping for (Moneyline, NBA)
[DEBUG] Missing table mapping for (Money